In [ ]:
import pickle

with open(r"pickles/model", "rb") as input_file:
    model = pickle.load(input_file)
    
with open(r"pickles/scaler", "rb") as input_file:
    scaler = pickle.load(input_file)

In [ ]:
import pandas as pd
import numpy as np

Combined_data = pd.read_csv('LosAngeles_2022.csv')
Combined_data.drop(['host_id', 'id', 'host_name','name', 'last_review', 'neighbourhood',
                    'license', 'number_of_reviews_ltm', 'price'], axis=1, inplace=True)
# print(Combined_data.columns)
# print(Combined_data.shape) ## 13 variables
# print(np.array(Combined_data.loc[0]))
new = pd.DataFrame(np.array(Combined_data.iloc[0,:])).T # 9 numbers
# print(new)
input = np.array(['Other Cities', 33.98301, -118.38607, 'Entire home/apt', 30, 2, 0.02, 1, 43])

# categorical: ['neighbourhood_group', 'room_type'] # 2
# numerical: ['latitude', 'longitude', 'price', 'minimum_nights', 'number_of_reviews',
#        'reviews_per_month', 'calculated_host_listings_count',
#        'availability_365', 'all_year_avail', 'low_avail', 'no_reviews'] # 11


# after onehot encoding 18 variables
# ['latitude', 'longitude', 'minimum_nights', 'number_of_reviews',
#        'reviews_per_month', 'calculated_host_listings_count',
#        'availability_365', 'all_year_avail', 'low_avail', 'no_reviews',
#        'neighbourhood_group_City of Los Angeles',
#        'neighbourhood_group_Other Cities',
#        'neighbourhood_group_Unincorporated Areas',
#        'neighbourhood_group_unknown', 
#        'room_type_Entire home/apt',
#        'room_type_Hotel room', 'room_type_Private room',
#        'room_type_Shared room']

# new.info()

# new['neighbourhood_group']

In [ ]:

new.columns = ['neighbourhood_group', 'latitude', 'longitude', 'room_type',
       'minimum_nights', 'number_of_reviews', 'reviews_per_month',
       'calculated_host_listings_count', 'availability_365']

for col in ['latitude', 'longitude', 'minimum_nights', 'number_of_reviews', 'reviews_per_month',
       'calculated_host_listings_count', 'availability_365']:
        new[col]=new[col].astype(np.float)
        
new['minimum_nights'] = np.log1p(new.loc[0,'minimum_nights'])

new['all_year_avail'] = new['availability_365']>353
new['low_avail'] = new['availability_365']< 12
new['no_reviews'] = new['reviews_per_month']==0

new['neighbourhood_group_City of Los Angeles'] = 1*(new['neighbourhood_group'] == 'City of Los Angeles')
new['neighbourhood_group_Other Cities'] = 1*(new['neighbourhood_group'] == 'Other Cities')
new['neighbourhood_group_Unincorporated Areas'] = 1*(new['neighbourhood_group'] == 'Unincorporated Areas')
new['neighbourhood_group_unknown'] = 1*(new['neighbourhood_group'] == 'unknown')
new['room_type_Entire home/apt'] = 1*(new['neighbourhood_group'] == 'Entire home/apt')
new['room_type_Hotel room'] = 1*(new['neighbourhood_group'] == 'Hotel room')
new['room_type_Private room'] = 1*(new['neighbourhood_group'] == 'Private room')
new['room_type_Shared room'] = 1*(new['neighbourhood_group'] == 'Shared room')

new.drop(['neighbourhood_group','room_type'], axis = 1, inplace = True)
print(new.shape)
print(new.columns)

# ['latitude', 'longitude', 'minimum_nights', 'number_of_reviews',
#        'reviews_per_month', 'calculated_host_listings_count',
#        'availability_365', 'all_year_avail', 'low_avail', 'no_reviews',
#        'neighbourhood_group_City of Los Angeles',
#        'neighbourhood_group_Other Cities',
#        'neighbourhood_group_Unincorporated Areas',
#        'neighbourhood_group_unknown', 'room_type_Entire home/apt',
#        'room_type_Hotel room', 'room_type_Private room',
#        'room_type_Shared room']


In [ ]:
new_scaled = scaler.transform(new)
new_scaled

In [ ]:
preds_scaled = model.predict(new_scaled)
preds = np.exp(preds_scaled[0])
preds

In [ ]:
def pipeline(new):
    # new = pd.DataFrame(arr).T # single entry
    new.columns = ['neighbourhood_group', 'latitude', 'longitude', 'room_type',
       'minimum_nights', 'number_of_reviews', 'reviews_per_month',
       'calculated_host_listings_count', 'availability_365']
    
    for col in ['latitude', 'longitude', 'minimum_nights', 'number_of_reviews', 'reviews_per_month',
       'calculated_host_listings_count', 'availability_365']:
        new[col]=new[col].astype(np.float)
    
    new['minimum_nights'] = np.log1p(new.loc[0,'minimum_nights'])

    new['all_year_avail'] = 1*(new['availability_365']>353)
    new['low_avail'] = 1*(new['availability_365']< 12)
    new['no_reviews'] = 1*(new['reviews_per_month']==0)

    new['neighbourhood_group_City of Los Angeles'] = 1*(new['neighbourhood_group'] == 'City of Los Angeles')
    new['neighbourhood_group_Other Cities'] = 1*(new['neighbourhood_group'] == 'Other Cities')
    new['neighbourhood_group_Unincorporated Areas'] = 1*(new['neighbourhood_group'] == 'Unincorporated Areas')
    new['neighbourhood_group_unknown'] = 1*(new['neighbourhood_group'] == 'unknown')
    
    new['room_type_Entire home/apt'] = 1*(new['neighbourhood_group'] == 'Entire home/apt')
    new['room_type_Hotel room'] = 1*(new['neighbourhood_group'] == 'Hotel room')
    new['room_type_Private room'] = 1*(new['neighbourhood_group'] == 'Private room')
    new['room_type_Shared room'] = 1*(new['neighbourhood_group'] == 'Shared room')

    new.drop(['neighbourhood_group','room_type'], axis = 1, inplace = True)

    new_scaled = scaler.transform(new)
    return new_scaled
 
# new = pd.DataFrame(input).T
# new.columns = ['neighbourhood_group', 'latitude', 'longitude', 'room_type',
#        'minimum_nights', 'number_of_reviews', 'reviews_per_month',
#        'calculated_host_listings_count', 'availability_365']
# # new['minimum_nights'] = np.log1p(new.loc[0,'minimum_nights'])
# new.loc[0,'minimum_nights']
# new.info()

# scaled = pipeline(input)
# preds_scaled = model.predict(scaled)
# preds = np.round(np.exp(preds_scaled[0]),4)
# preds
